In [18]:
import operator as op
from scipy.stats import poisson, binom
from math import factorial
import numpy as np
from matplotlib import pyplot as plt
import scipy.misc as spm
import time 
import numpy as np

params = {'a_min': 75, 'a_max': 90, 'b_min': 500,
          'b_max': 600, 'p_1': 0.1, 'p_2': 0.01, 'p_3': 0.3}

In [33]:
def Ea(params):
    a_max = params['a_max']
    a_min = params['a_min']
    
    return (a_max + a_min)/2.0

def Da (params):
    a_max = params['a_max']
    a_min = params['a_min']
    
    return ((a_max - a_min + 1) - 1)**2/12.0

def Db (params):
    b_min = params['b_min']
    b_max = params['b_max']
    
    return ((b_max - b_min + 1) - 1)**2/12.0

def Eb(params):
    b_min = params['b_min']
    b_max = params['b_max']
    
    return (b_max + b_min)/2.0

In [6]:
print 'Expectation a: Ea = {}, variance a: Da = {}'.format(Ea(params), Da(params))
print 'Expectation b: Eb = {}, variance b: Db = {}'.format(Eb(params), Db(params))

Expectation a: Ea = 82.5, variance a: Da = 18.75
Expectation b: Eb = 550.0, variance b: Db = 833.333333333


In [34]:
def pc_a(a, params, model=2):
    
    a_max = params['a_max']
    b_min = params['b_min']
    b_max = params['b_max']
    p1 = params['p_1']
    p2 = params['p_2']
    
    b = np.arange(b_min, b_max + 1)   
    c = np.arange(a_max + b_max + 1)

    
    if model == 2:
        
        pois_a = poisson.pmf(range(a_max + 1), a * p1)
        pois_b = poisson.pmf(range(b_max + 1), b.reshape(-1, 1) * p2)
        p = np.convolve(pois_a, np.sum(pois_b, axis=0))
         
    
    if model == 1:
        
        binom_a = binom.pmf(range(a_max + 1), a,  p1)
        binom_b = binom.pmf(range(b_max + 1), b.reshape(-1, 1),  p2)
        p = np.convolve(binom_a, np.sum(binom_b, axis=0))

    p = p / sum(p)
    return p, c

In [40]:
E_a = int(round(Ea(params)))

for i in [1,2]:
    
    c_a = pc_a(E_a, params, model = i)
    E_ca = np.dot(c_a[1], c_a[0])
    D_ca = np.dot((c_a[1] - E_ca)**2, c_a[0])
    print 'Model', i,'. Expectation c: Ec = {}, variance c: Dc = {}'.format(E_ca, D_ca)

Model 1 . Expectation c: Ec = 13.8, variance c: Dc = 13.0
Model 2 . Expectation c: Ec = 13.8, variance c: Dc = 13.885


In [41]:
def pc_b(b, params, model=2):
    
    a_min = params['a_min']
    a_max = params['a_max']
    b_max = params['b_max']
    p1 = params['p_1']
    p2 = params['p_2']
    
    a = np.arange(a_min, a_max + 1)
    c = np.arange(a_max + b_max + 1)


    if model == 2:
        pois_a = poisson.pmf(range(a_max + 1), a.reshape(-1, 1) * p1)
        pois_b = poisson.pmf(range(b_max + 1), b * p2)
        p = np.convolve(np.sum(pois_a, axis=0), pois_b)
         
    if model == 1:
        binom_a = binom.pmf(range(a_max + 1), a.reshape(-1, 1),  p1)
        binom_b = binom.pmf(range(b_max + 1), b,  p2)
        p = np.convolve(np.sum(binom_a, axis=0), binom_b)
    
    p = p / sum(p)
    return p, c

In [74]:
E_b = int(round(Eb(params)))

for i in [1,2]:
    
    c_b = pc_b(E_b, params, model = i)
    E_cb = np.dot(c_b[1], c_b[0])
    D_cb = np.dot((c_b[1] - E_cb)**2, c_b[0])
    print 'Model', i,'. Expectation c: Ec = {}, variance c: Dc = {}'.format(E_cb, D_cb)

Model 1 . Expectation c: Ec = 13.75, variance c: Dc = 13.0825
Model 2 . Expectation c: Ec = 13.75, variance c: Dc = 13.9625


In [44]:
def pc_ab(a, b, params, model=2):  
    a_max = params['a_max']
    b_max = params['b_max']
    p1 = params['p_1']
    p2 = params['p_2']
    c = np.arange(a + b + 1)
    
    if model == 2:
        p = poisson.pmf(range(a + b + 1), a * p1 + b * p2)
         
    if model == 1:
        binom_a = binom.pmf(range(a + 1), a, p1)
        binom_b = binom.pmf(range(b + 1), b, p2)
        p = np.convolve(binom_a, binom_b)
    
    p = p / sum(p)
    return p, c

In [45]:
E_a = int(round(Ea(params)))
E_b = int(round(Eb(params)))

for i in [1,2]:
    
    c_ab = pc_ab(E_a, E_b, params, model = i)
    Ec_ab = np.dot(c_ab[1], c_ab[0])
    Dc_ab = np.dot((c_ab[1] - Ec_ab)**2, c_ab[0])
    print 'Model', i,'. Expectation c: Ec = {}, variance c: Dc = {}'.format(Ec_ab, Dc_ab)

Model 1 . Expectation c: Ec = 13.8, variance c: Dc = 12.915
Model 2 . Expectation c: Ec = 13.8, variance c: Dc = 13.8


In [46]:
def pc(params, model=2):
    
    a_min = params['a_min']
    a_max = params['a_max']
    b_min = params['b_min']
    b_max = params['b_max']
    p1 = params['p_1']
    p2 = params['p_2']

    a = np.arange(a_min, a_max + 1)
    b = np.arange(b_min, b_max + 1)
    c = np.arange(a_max + b_max + 1)


    if model == 2:
        pois_a = poisson.pmf(range(a_max + 1), a.reshape(-1, 1) * p1)
        pois_b = poisson.pmf(range(b_max + 1), b.reshape(-1, 1) * p2)
        p = np.convolve(np.sum(pois_a, axis=0), np.sum(pois_b, axis=0))
        p = p / sum(p)

    if model == 1:
        binom_a = binom.pmf(range(a_max + 1), a.reshape(-1, 1), p1)
        binom_b = binom.pmf(range(b_max + 1), b.reshape(-1, 1), p2)
        p = np.convolve(np.sum(binom_a, axis=0), np.sum(binom_b, axis=0))
        p = p / sum(p)

    return p, c

In [48]:
for i in [1,2]:
    
    p_c = pc(params, model = i)
    Ec = np.dot(p_c[1], p_c[0])
    Dc = np.dot((p_c[1] - Ec)**2, p_c[0])
    print 'Model', i,'. Expectation c: Ec = {}, variance c: Dc = {}'.format(Ec, Dc)

Model 1 . Expectation c: Ec = 13.75, variance c: Dc = 13.1675
Model 2 . Expectation c: Ec = 13.75, variance c: Dc = 14.0475


In [53]:
def pd(params, model=2):
    
    p3 = params['p_3']
    a_max = params['a_max']
    b_max = params['b_max']
    
    p_c, c = pc(params, model)
    
    d = np.arange(2 * c.max() + 1)
    p = []
    
    binom_c = binom.pmf(range(c.max() + 1), c.reshape(-1, 1), p3)
    D = np.multiply(binom_c, np.repeat(p_c, len(c)).reshape(len(c), -1))
    for i in range(-c.max(), c.max() + 1):
        p.append(np.trace(np.flipud(D), i))
    
    p = np.array(p)
    p = p / sum(p)
    
    return p, d

In [57]:
for i in [1,2]:
    
    p_d = pd(params, model = i)
    Ed = np.dot(p_d[1], p_d[0])
    Dd = np.dot((p_d[1] - Ed)**2, p_d[0])
    print 'Model', i,'. Expectation c: Ed = {}, variance c: Dd = {}'.format(Ed, Dd)

Model 1 . Expectation c: Ed = 17.875, variance c: Dd = 25.140575
Model 2 . Expectation c: Ed = 17.875, variance c: Dd = 26.627775


In [56]:
def pc_d(d, params, model=2):
    
    p3 = params['p_3']
    a_max = params['a_max']
    b_max = params['b_max']
    
    
    p_c, c = pc(params, model=model)

    binom_c = binom.pmf(range(c.max() + 1), c.reshape(-1, 1), p3) 
    
    D = np.multiply(binom_c, np.repeat(p_c, len(c)).reshape(len(c), -1))
    
    p = np.diagonal(np.flipud(D), d - c.max())[::-1]
    p = np.concatenate((np.zeros(max(d-c.max(), 0)), p))
    p = np.concatenate((p, np.zeros(len(c) - len(p))))

    p = p / sum(p)
    
    return p, c

In [59]:
for i in [1,2]:
    
    p_d = pd(params, model = i)
    Ed = int(round(np.dot(p_d[1], p_d[0])))
    
    p_cd = pc_d(Ed, params, model = i)
    Ec_d = np.dot(p_cd[1], p_cd[0])
    Dc_d = np.dot((p_cd[1] - Ec_d)**2, p_cd[0])
    print 'Model', i,'. Expectation c: Ec = {}, variance c: Dc = {}'.format(Ec_d, Dc_d)

Model 1 . Expectation c: Ec = 13.895970599, variance c: Dc = 1.53358170067
Model 2 . Expectation c: Ec = 13.8938338922, variance c: Dc = 1.5439433291


In [72]:

def pc_abd(a, b, d, params, model=2):
    
    a_min = params['a_min']
    a_max = params['a_max']
    b_min = params['b_min']
    b_max = params['b_max']
    p1 = params['p_1']
    p2 = params['p_2']
    p3 = params['p_3']
    
    c = np.arange(min(a + b, d) + 1)
    p_cab = pc_ab(a, b, params, model)[0][:c.max() + 1]
    binom_c = binom.pmf(range(c.max() + 1), c.reshape(-1, 1), p3) 
    
    D = np.multiply(binom_c, np.repeat(p_cab, len(c)).reshape(len(c), -1))
    p = np.diagonal(np.flipud(D), d - c.max())[::-1]
        
    p = p / sum(p)
    
    return p, c

In [73]:
E_a = int(round(Ea(params)))
E_b = int(round(Eb(params)))

for i in [1,2]:
    
    p_d = pd(params, model = i)
    Ed = int(round(np.dot(p_d[1], p_d[0])))
    
    p_cabd = pc_abd(E_a, E_b, Ed, params, model = i)
    Ec_abd = np.dot(p_cabd[1], p_cabd[0])
    Dc_abd = np.dot((p_cabd[1] - Ec_abd)**2, p_cabd[0])
    print 'Model', i,'. Expectation c: Ec = {}, variance c: Dc = {}'.format(Ec_abd, Dc_abd)

Model 1 . Expectation c: Ec = 13.9027555944, variance c: Dc = 1.53014020852
Model 2 . Expectation c: Ec = 13.900175167, variance c: Dc = 1.54088384958


#### Task 4

In [71]:
a = int(round(Ea(params)))
b = int(round(Eb(params)))

# p(c)
for i in [1,2]:

    start_time = time.time()
    pc(params)
    print 'time p(c) model', i, ': ',  round(time.time() - start_time,4), 'sec'


print '\n'
# p(c|a)    

for i in [1,2]:

    start_time = time.time()
    pc_a(a, params)
    print 'time p(c|a) model', i, ': ',  round(time.time() - start_time,4), 'sec'
    
print '\n'    
# p(c|b)

for i in [1,2]:

    start_time = time.time()
    pc_b(b, params)
    print 'time p(c|b) model', i, ': ',  round(time.time() - start_time,4), 'sec'

print '\n'
# p(c|ab)
for i in [1,2]:

    start_time = time.time()
    pc_ab(a,b,params)
    print 'time p(c|ab) model', i, ': ',  round(time.time() - start_time,4), 'sec'


print '\n'
# p(c|d)
for i in [1,2]:
    
    p_d = pd(params, model = i)
    Ed = int(round(np.dot(p_d[1], p_d[0])))
    start_time = time.time()
    pc_d(Ed, params)
    print 'time p(c|d) model', i, ': ',  round(time.time() - start_time,4), 'sec'

print '\n'
#p(c|abd)
for i in [1,2]:
    
    p_d = pd(params, model = i)
    Ed = int(round(np.dot(p_d[1], p_d[0])))
    start_time = time.time()
    pc_abd(a, b, Ed, params)
    print 'time p(c|abd) model', i, ': ',  round(time.time() - start_time,4), 'sec'

print '\n'
#p(d)
for i in [1,2]:
    
    start_time = time.time()
    p_d = pd(params, model = i)
    print 'time p(d) model', i, ': ',  round(time.time() - start_time,4), 'sec'

time p(c) model 1 :  0.0275 sec
time p(c) model 2 :  0.0258 sec


time p(c|a) model 1 :  0.0259 sec
time p(c|a) model 2 :  0.0339 sec


time p(c|b) model 1 :  0.0024 sec
time p(c|b) model 2 :  0.002 sec


time p(c|ab) model 1 :  0.0011 sec
time p(c|ab) model 2 :  0.0012 sec


time p(c|d) model 1 :  0.2426 sec
time p(c|d) model 2 :  0.2404 sec


time p(c|abd) model 1 :  0.0025 sec
time p(c|abd) model 2 :  0.0024 sec


time p(d) model 1 :  0.2835 sec
time p(d) model 2 :  0.2757 sec
